In [3]:
import numpy as np
import yfinance as yf
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense , LSTM , Dropout

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
!pip install yfinance

In [5]:
# Define the ticker symbol for Apple
ticker_symbol = "AAPL"

# Define the date range for 2013 until 2018
start_date = "2013-01-01"
end_date = "2018-01-01"

# Download the stock data using yfinance
apple_stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

# Reset index to have the date as a column
apple_stock_data.reset_index(inplace=True)

# Save the data to a CSV file
csv_file_name = "stock.csv"
apple_stock_data.to_csv(csv_file_name, index=False)

print("Stock data saved to", csv_file_name)


[*********************100%%**********************]  1 of 1 completed
Stock data saved to stock.csv


In [6]:
# Load the data from the CSV file
csv_file_name = "stock.csv"
apple_stock_data = pd.read_csv(csv_file_name)

# Create a line chart using Plotly Express
fig = px.line(apple_stock_data, x="Date", y="Close", title="Apple Stock Prices in 2013 - 2017")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Close Price")

# Show the plot
fig.show()

In [7]:
apple_stock_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2013-01-02,19.779285,19.821428,19.343929,19.608213,16.791183,560518000
1,2013-01-03,19.567142,19.631071,19.321428,19.360714,16.579241,352965200
2,2013-01-04,19.177500,19.236786,18.779642,18.821428,16.117434,594333600
3,2013-01-07,18.642857,18.903570,18.400000,18.710714,16.022625,484156400
4,2013-01-08,18.900356,18.996071,18.616072,18.761070,16.065746,458707200


In [8]:
# data = apple_stock_data.["Open"][:,1:2].values
data = apple_stock_data.iloc[:,1:2].values

In [9]:
data

array([[19.77928543],
       [19.56714249],
       [19.17749977],
       ...,
       [42.52500153],
       [42.75      ],
       [42.63000107]])

In [10]:
type(data)

numpy.ndarray

In [11]:
len(data)

1259

# Scaling the dataset

In [12]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [13]:
data_scaled

array([[0.19795893],
       [0.19086893],
       [0.17784673],
       ...,
       [0.95814041],
       [0.96566006],
       [0.96164959]])

In [14]:
data_scaled.shape

(1259, 1)

In [15]:
feature_set=[]#X
labels=[]#y

for i in range (60,1259):
    feature_set.append(data_scaled[i-60:i,0])
    labels.append(data_scaled[i,0])

In [16]:
type(feature_set)

list

In [17]:
feature_set = np.array(feature_set)
labels = np.array(labels)

In [18]:
feature_set.shape

(1199, 60)

In [19]:
feature_set[1]

array([0.19086893, 0.17784673, 0.15997849, 0.16858436, 0.16057528,
       0.16779659, 0.15878493, 0.13691808, 0.13169014, 0.12732155,
       0.1460253 , 0.13195271, 0.1391621 , 0.14423489, 0.08597518,
       0.07605636, 0.05951303, 0.08418477, 0.08239436, 0.08237051,
       0.08491287, 0.07870612, 0.06693722, 0.08176175, 0.0898544 ,
       0.10268561, 0.10566959, 0.10926234, 0.09458104, 0.09137026,
       0.09653853, 0.08728814, 0.08321794, 0.06926472, 0.07314396,
       0.07863453, 0.06666267, 0.07216516, 0.06693722, 0.0597159 ,
       0.04754117, 0.03999762, 0.05555025, 0.04360229, 0.04992839,
       0.0498687 , 0.05685129, 0.04831704, 0.05354498, 0.05963236,
       0.06383385, 0.08537833, 0.08289571, 0.07430177, 0.07950587,
       0.09157316, 0.09246833, 0.08174983, 0.07382432, 0.06437096])

In [20]:
labels[1]

0.0473024735830167

In [21]:
feature_set.shape

(1199, 60)

In [22]:
feature_set=np.reshape(feature_set,(feature_set.shape[0],feature_set.shape[1],1))

In [23]:
feature_set.shape

(1199, 60, 1)

# Deep Learning Model

In [24]:
model = Sequential()

In [25]:
model.add(LSTM(units=100,return_sequences=True,input_shape=(feature_set.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100,return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(units=1))

In [26]:
model.compile(optimizer='adam',loss='mean_squared_error')

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 100)           40800     
                                                                 
 dropout (Dropout)           (None, 60, 100)           0         
                                                                 
 lstm_1 (LSTM)               (None, 60, 100)           80400     
                                                                 
 dropout_1 (Dropout)         (None, 60, 100)           0         
                                                                 
 lstm_2 (LSTM)               (None, 60, 100)           80400     
                                                                 
 dropout_2 (Dropout)         (None, 60, 100)           0         
                                                                 
 lstm_3 (LSTM)               (None, 60, 100)           8

In [28]:
model.fit(feature_set,labels,epochs=100,batch_size=32)

Epoch 1/100
38/38 [==============================] - 12s 17ms/step - loss: 0.0934
Epoch 2/100
38/38 [==============================] - 1s 18ms/step - loss: 0.0652
Epoch 3/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0636
Epoch 4/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0622
Epoch 5/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0619
Epoch 6/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0617
Epoch 7/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0613
Epoch 8/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0605
Epoch 9/100
38/38 [==============================] - 1s 16ms/step - loss: 0.0604
Epoch 10/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0601
Epoch 11/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0604
Epoch 12/100
38/38 [==============================] - 1s 17ms/step - loss: 0.0600
Epoch 13/100
38/38 [====